# Full GMM procedure: parameters to entropy to lambda to (finally) moment errors

This notebooks aims to have a complete GMM estimation procedure.
It goes:

+ 1) the structural parameters and data $(\theta, Z)$
+ 2) Entropy of initial lambdas ($\lambda_0$) and the expected value of ($\lambda_0$)  is a function of data and structural parameters. $H = f( \theta' Z)$, $E[B] = g(\theta' Z)$
+ 3) ($\lambda_0$)  is a vector that is a function of entropy and expected value. $\lambda_0 = K(H, E[B])$
+ 4) The simulated moments is a function of ($\lambda_0$). $M = L(\lambda_0)$. Kalman filter might help me do something better :D

This involves two steps

+ I) Numerical prrocedure to get ($\lambda_0$) from $(\theta_{try}, Z)$, through $(H, E[B])$.
+ II) (Numerical?) procedure to get simulated moments from ($\lambda_0$).


## I) Numerical procedure to get ($\lambda_0$) from $(\theta_{try}, Z)$

through $(H, E[B])$. See `unique_lambdas_from_H.ipynb` for more details

It makes sense to have a multiple starting point thingy.

In [13]:
def force_sum_to_1(x):
    """
    Forces lambdas to sum to 1
    Only works for three lambdas
    """
    return [x[0], x[1], 1 - x[0] - x[1]]


def force_sum_to_1_(x):
    return np.hstack([x, 1-x.sum()])

x = np.array([0.3, 0.5])
print(force_sum_to_1(x))
force_sum_to_1_(x)

[0.3, 0.5, 0.19999999999999996]


array([0.3, 0.5, 0.2])

In [15]:
import numpy as np
from scipy import optimize

#Parameters
Eβ = 1.2
βs = [0.7, 1.1, 1.5] # Corresponding to each lambda
H = 0.8

def my_entropy(p):
    return -np.sum(p * np.log(p))


def force_sum_to_1(x):
    """
    Forces lambdas to sum to 1
    (although last element might be negative)
    """
    return np.hstack([x, 1-x.sum()])


def logit(p):
    return np.log(p / (1 - p))


def reparam_lambdas(x):
    """ inverse logit. Forces the lambdas to be within 0 and 1"""
    return np.e**x / (1 + np.e**x)


def fun(x, βs, Eβ, H):
    """
    x: deep parameters
    Eβ, H: the objectives
    βs: fixed constant of the model
    """
    lambdas = force_sum_to_1(reparam_lambdas(x))
    return [my_entropy(lambdas) - H,
            np.dot(βs, lambdas) - Eβ]

def jac(x, βs):
    """
    Jacobian for reparametrization of lambdas.
    Code for only three lambdas
    """
    # Derivatives wrt to H
    block = np.log((1 - np.e ** (x[0] + x[1])) / (np.e ** (x[0]) + np.e ** (x[1]) + np.e ** (x[0] + x[1]) + 1))
    num0 = (-np.log(np.e ** x[0] / (np.e ** x[0] + 1)) + block) * np.e ** x[0]
    den0 = np.e ** (2 * x[0]) + 2 * np.e ** (x[0]) + 1
    num1 = (-np.log(np.e ** x[1] / (np.e ** x[1] + 1)) + block) * np.e ** x[1]
    den1 = np.e ** (2 * x[1]) + 2 * np.e ** (x[1]) + 1

    dh_dx = np.array([num0 / den0, num1 / den1])

    # Derivatives wrt E[B]
    deb_0 = ((βs[0] - βs[2]) * np.e ** (-x[0])) / (1 + np.e ** (-x[0])) ** 2
    deb_1 = ((βs[1] - βs[2]) * np.e ** (-x[1])) / (1 + np.e ** (-x[1])) ** 2
    deb_dx = np.array([deb_0, deb_1])

    return np.array([dh_dx, deb_dx])

def relative_error(true, solution):
    """
    Average relative error to discriminate solutions
    """
    return np.mean(2*np.abs((true - solution) / (true + solution)))


def fun_(x):
    return fun(x, βs=βs, Eβ=Eβ, H=H)


def jac_(x):
    return jac(x, βs=βs)

sol = optimize.root(fun_, logit(np.array([0.1, 0.5])), jac=jac_)
print(sol.message)

lambdas_sol = force_sum_to_1(reparam_lambdas(sol.x))
print(lambdas_sol)
print("target values for objective: ", H, Eβ)
print(f"Solution values for objective: {my_entropy(lambdas_sol):.2f},"
      + f" {np.dot(βs, lambdas_sol):.2f}")

The solution converged.
[0.05273249 0.64453502 0.30273249]
target values for objective:  0.8 1.2
Solution values for objective: 0.80, 1.20


In [60]:
starting_points = np.array([[0.1, 0.2], [0.3, 0.3],
                           [0.25, 0.45], [0.45, 0.25]])

def ndprint(a, format_string='{0:.3f}'):
    print([format_string.format(v,i) for i,v in enumerate(a)])

target = np.array([H, Eβ])
print("target values for objective: ", target[0], target[1])
print("  ")
for x0 in starting_points:
    print(f"x0: {x0}")
    sol = optimize.root(fun, logit(x0), jac=jac)
    lambdas_sol = x_to_lambdas(x_to_p(sol.x))
    ndprint(np.array(lambdas_sol))
    solution = np.array([my_entropy(lambdas_sol), np.dot(βs, lambdas_sol)])
    print(f"Solution values for objective: {solution[0]:.2f} ,"
      + f" {solution[1]:.2f}")
    print(f"Error: {relative_error(target, solution):10.1e}")
    print("=========================================")
    print(" ")

target values for objective:  0.8 1.2
  
x0: [0.1 0.2]
['0.355', '0.040', '0.605']
Solution values for objective: 0.80 , 1.20
Error:    2.6e-14
 
x0: [0.3 0.3]
['0.355', '0.040', '0.605']
Solution values for objective: 0.80 , 1.20
Error:    1.5e-13
 
x0: [0.25 0.45]
['0.053', '0.645', '0.303']
Solution values for objective: 0.80 , 1.20
Error:    1.1e-14
 
x0: [0.45 0.25]
['0.355', '0.040', '0.605']
Solution values for objective: 0.80 , 1.20
Error:    1.1e-12
 


In [31]:
type(lambdas_sol)

list

In [7]:
import numpy as np
#x -> [x[0], x[1], 1 - x[0] - x[1]]
x = np.array([0.4, 0.8, 0.3])
z = np.hstack([x, 1-x.sum()])
z


array([ 0.4,  0.8,  0.3, -0.5])

## II) (Numerical?) procedure to get simulated moments from ($\lambda_0$).

See `gmm_example.py`

In [4]:
import numpy as np
import dill
import pandas as pd
from scipy import optimize as opt
import time
import sys
sys.path.append('../../')
import src


#Load policy and value function
#####################
file_n = "2018-10-5vfi_dict.dill"
with open('../../data/' + file_n, 'rb') as file:
    data_d = dill.load(file)

lambdas = src.generate_simplex_3dims(n_per_dim=data_d['n_of_lambdas_per_dim'])
price_grid = np.linspace(data_d['min_price'], data_d['max_price'])

policy = data_d['policy']
valueF = data_d['valueF']
lambdas_ext = src.generate_simplex_3dims(n_per_dim=
                                         data_d['n_of_lambdas_per_dim'])

#Interpolate policy (level price). valueF is already a function
policyF = src.interpolate_wguess(lambdas_ext, policy)


# Simulation parameters \
########################
σerror= 0.005 #0.01
Nfirms = 300
time_periods = 40
min_periods= 3

#Suitable for logistic
β10, β11 = -2., 3.
β20, β21 = 0.03, -2.
betas = [β10, β11, β20, β21]

#GMM parameters
maxiters = 3


def lambda_0(x, prior_shock) -> np.ndarray:
    """
    Generate a vector of lambdas on the observables x
    """
    return src.from_theta_to_lambda0(x, θ=betas, prior_shock=prior_shock)


xs = np.abs(np.random.normal(0, 0.18, size=Nfirms))
prior_shocks = src.gen_prior_shocks(Nfirms, σerror=σerror)

dmd_shocks = src.generate_dmd_shocks(n=Nfirms, t=time_periods, dmd_σϵ=src.const.σ_ɛ)

df = src.simulate_all_firms(Nfirms, valueF, policyF, xs, θ=betas,
                   dmd_shocks=dmd_shocks, prior_shocks=prior_shocks)

std_devs = (df.groupby('firm').level_prices.rolling(window=4, min=3)
            .std().reset_index()
            .rename(columns={'level_1': 't',
                            'level_prices': 'std_dev_prices'}))

df = pd.merge(df, std_devs, on=['firm', 't'], how='left')

mean_std_observed_prices = df.groupby('t').std_dev_prices.mean()[min_periods:]


def error_w_data(θ) -> float:
    return src.gmm_error(θ, policyF, xs,
                      mean_std_observed_prices=mean_std_observed_prices, df=df,
                                 prior_shocks=prior_shocks, min_periods=min_periods)

print("Preprocessing done. Now starting optimization")

start = time.time()

optimi = opt.differential_evolution(error_w_data, [(-2.5, 0.5), (3., 3.2),
                                                   (-0.5, 0.2), (-3, 1)],
                                    maxiter=maxiters)

Preprocessing done. Now starting optimization


## III) Putting step 1 and 2 together

get_lambdas_from_x uses lambda_0(x, prior_shock)??


I think I only need to change `src.from_theta_to_lambda0` to include the intermediate step
through H, E[B]